<a href="https://colab.research.google.com/github/yjy9500/Etri_project/blob/master/Final_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==1.14
#tensorflow 1.14 version 설치

     |████████████████████████████████| 109.2MB 91kB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 491kB 49.1MB/s 
     |████████████████████████████████| 3.2MB 44.4MB/s 
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [ ]:
!pip uninstall scipy
!pip install scipy==1.1.0
#scipy version 하향 및 1.1.0version 설치

Uninstalling scipy-1.4.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/scipy-1.4.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/scipy/*
Proceed (y/n)? y
  Successfully uninstalled scipy-1.4.1
     |████████████████████████████████| 31.2MB 148kB/s 
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [ ]:
!pip install module
#module 라이브러리 설치

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
#google drive 사용 허가 및 승인

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import os 
os.chdir('/content/gdrive/My Drive')
#본인의 드라이브 사용시 /My Drive 의 내용을 본인의 폴더 directory로 변경 가능
!pwd
#pwd 명령어를 통해 현재 디렉토리의 절대경로를 확인 가능

/content/gdrive/My Drive


In [ ]:
from __future__ import division
import tensorflow as tf
import tensorflow.contrib.slim as slim
import math
import numpy as np
import pprint
import scipy.misc
import copy
#라이브러리 Import 하는 명령어

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
#판별기 함수 생성
def discriminator(image, options, reuse=False, name="discriminator"):

    with tf.variable_scope(name):
        # image is 256 x 256 x input_c_dim
        if reuse:
            tf.get_variable_scope().reuse_variables()
        else:
            assert tf.get_variable_scope().reuse is False

        h0 = lrelu(conv2d(image, options.df_dim, name='d_h0_conv'))
        # h0 is (128 x 128 x self.df_dim)
        h1 = lrelu(instance_norm(conv2d(h0, options.df_dim*2, name='d_h1_conv'), 'd_bn1'))
        # h1 is (64 x 64 x self.df_dim*2)
        h2 = lrelu(instance_norm(conv2d(h1, options.df_dim*4, name='d_h2_conv'), 'd_bn2'))
        # h2 is (32x 32 x self.df_dim*4)
        h3 = lrelu(instance_norm(conv2d(h2, options.df_dim*8, s=1, name='d_h3_conv'), 'd_bn3'))
        # h3 is (32 x 32 x self.df_dim*8)
        h4 = conv2d(h3, 1, s=1, name='d_h3_pred')
        # h4 is (32 x 32 x 1)
        return h4

#생성기 함수 생성
def generator_resnet(image, options, reuse=False, name="generator"):

    with tf.variable_scope(name):
        # image is 256 x 256 x input_c_dim
        if reuse:
            tf.get_variable_scope().reuse_variables()
        else:
            assert tf.get_variable_scope().reuse is False

        def residule_block(x, dim, ks=3, s=1, name='res'):
            p = int((ks - 1) / 2)
            y = tf.pad(x, [[0, 0], [p, p], [p, p], [0, 0]], "REFLECT")      # padding은 양옆 위아래로 1씩만.

            # instance_norm은 nomalization.
            y = instance_norm(conv2d(y, dim, ks, s, padding='VALID', name=name+'_c1'), name+'_bn1')
            y = tf.pad(tf.nn.relu(y), [[0, 0], [p, p], [p, p], [0, 0]], "REFLECT")
            y = instance_norm(conv2d(y, dim, ks, s, padding='VALID', name=name+'_c2'), name+'_bn2')
            return y + x


        c0 = tf.pad(image, [[0, 0], [3, 3], [3, 3], [0, 0]], "REFLECT")     # 위 아래 가로 세로로 패딩 3씩.
        c1 = tf.nn.relu(instance_norm(conv2d(c0, options.gf_dim, 7, 1, padding='VALID', name='g_e1_c'), 'g_e1_bn'))
        c2 = tf.nn.relu(instance_norm(conv2d(c1, options.gf_dim*2, 3, 2, name='g_e2_c'), 'g_e2_bn'))
        c3 = tf.nn.relu(instance_norm(conv2d(c2, options.gf_dim*4, 3, 2, name='g_e3_c'), 'g_e3_bn'))        # 64 *64

        # define G network with 9 resnet blocks
        # 총 cnn 18층.
        r1 = residule_block(c3, options.gf_dim*4, name='g_r1')
        r2 = residule_block(r1, options.gf_dim*4, name='g_r2')
        r3 = residule_block(r2, options.gf_dim*4, name='g_r3')
        r4 = residule_block(r3, options.gf_dim*4, name='g_r4')
        r5 = residule_block(r4, options.gf_dim*4, name='g_r5')
        r6 = residule_block(r5, options.gf_dim*4, name='g_r6')
        r7 = residule_block(r6, options.gf_dim*4, name='g_r7')
        r8 = residule_block(r7, options.gf_dim*4, name='g_r8')
        r9 = residule_block(r8, options.gf_dim*4, name='g_r9')

        d1 = deconv2d(r9, options.gf_dim*2, 3, 2, name='g_d1_dc')
        d1 = tf.nn.relu(instance_norm(d1, 'g_d1_bn'))
        d2 = deconv2d(d1, options.gf_dim, 3, 2, name='g_d2_dc')
        d2 = tf.nn.relu(instance_norm(d2, 'g_d2_bn'))
        d2 = tf.pad(d2, [[0, 0], [3, 3], [3, 3], [0, 0]], "REFLECT")

        # 위에 패딩을 해놨고 7 by 7 필터사용.
        pred = tf.nn.tanh(conv2d(d2, options.output_c_dim, 7, 1, padding='VALID', name='g_pred_c'))



        return pred

In [ ]:
from scipy.misc import imread, imresize, imsave
def abs_criterion(in_, target):
    return tf.reduce_mean(tf.abs(in_ - target))

def mae_criterion(in_, target):
    return tf.reduce_mean((in_-target)**2)

def sce_criterion(logits, labels):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels))

# 이미지를 풀링
class ImagePool(object):
    def __init__(self, maxsize=50):
        self.maxsize = maxsize
        self.num_img = 0
        self.images = []

    def __call__(self, image):
        if self.maxsize <= 0:
            return image
        if self.num_img < self.maxsize:
            self.images.append(image)
            self.num_img += 1
            return image
        if np.random.rand() > 0.5:
            idx = int(np.random.rand()*self.maxsize)
            tmp1 = copy.copy(self.images[idx])[0]
            self.images[idx][0] = image[0]
            idx = int(np.random.rand()*self.maxsize)
            tmp2 = copy.copy(self.images[idx])[1]
            self.images[idx][1] = image[1]
            return [tmp1, tmp2]
        else:
            return image

"""=========== 모델 만들 때 필요한 함수 =============="""

def instance_norm(input, name="instance_norm"):
    with tf.variable_scope(name):
        depth = input.get_shape()[3]            # 채널 수.

        # 평균 1, 표준편차 0.02 nomal distribution, 즉 채널 수 갯수만큼 변수 생성.
        scale = tf.get_variable("scale", [depth], initializer=tf.random_normal_initializer(1.0, 0.02, dtype=tf.float32))

        #채널 수 갯수 만큼 변수 생성.
        offset = tf.get_variable("offset", [depth], initializer=tf.constant_initializer(0.0))

        mean, variance = tf.nn.moments(input, axes=[1,2], keep_dims=True)       # 가로, 세로에 대해서 평균과 분산을 구하기.
        epsilon = 1e-5
        inv = tf.rsqrt(variance + epsilon)
        normalized = (input-mean)*inv
        return scale*normalized + offset
#convolution 함수
def conv2d(input_, output_dim, ks=4, s=2, stddev=0.02, padding='SAME', name="conv2d"):
    with tf.variable_scope(name):       # 앞에 conv2d가 붙음.
        # output_dim == 필터수. ks == 커널 사이즈.(4 x4) s==스트라이드, 활성 함수 없음.
        # 이미지 사이즈 가로 세로 각각 0.5배
        return slim.conv2d(input_, output_dim, ks, s, padding=padding, activation_fn=None,
                            weights_initializer=tf.truncated_normal_initializer(stddev=stddev),
                            biases_initializer=None)
#deconvolution 함수
def deconv2d(input_, output_dim, ks=4, s=2, stddev=0.02, name="deconv2d"):
    with tf.variable_scope(name):
        # 이미지 사이즈 가로 세로 각각 2배됨.
        return slim.conv2d_transpose(input_, output_dim, ks, s, padding='SAME', activation_fn=None,
                                    weights_initializer=tf.truncated_normal_initializer(stddev=stddev),
                                    biases_initializer=None)

def lrelu(x, leak=0.2, name="lrelu"):
    return tf.maximum(x, leak*x)

"""======================== 여기는 모델외 모듈들 .=================================="""

_imread = scipy.misc.imread

def load_test_data(image_path, fine_size=256):
    img = imread(image_path)
    img = scipy.misc.imresize(img, [fine_size, fine_size])
    img = img/127.5 - 1         # -1에서 1사이로 맞추려고.
    return img

def load_train_data(image_path, load_size=286, fine_size=256, is_testing=False):
    img_A = imread(image_path[0])
    img_B = imread(image_path[1])

    if not is_testing:  # 훈련 중
        img_A = scipy.misc.imresize(img_A, [load_size, load_size])
        img_B = scipy.misc.imresize(img_B, [load_size, load_size])
        h1 = int(np.ceil(np.random.uniform(1e-2, load_size-fine_size))) # 1에서 load_size-fine_size 사이의 값 하나.
        w1 = int(np.ceil(np.random.uniform(1e-2, load_size-fine_size)))
        img_A = img_A[h1:h1+fine_size, w1:w1+fine_size]         # 이미지를 랜덤하게 컷팅... why??
        img_B = img_B[h1:h1+fine_size, w1:w1+fine_size]

        if np.random.random() > 0.5:                # 50프로 확률로 이미지 뒤집기
            img_A = np.fliplr(img_A)
            img_B = np.fliplr(img_B)

    else:   # 테스트 중!
        img_A = scipy.misc.imresize(img_A, [fine_size, fine_size])
        img_B = scipy.misc.imresize(img_B, [fine_size, fine_size])

    img_A = img_A/127.5 - 1.
    img_B = img_B/127.5 - 1.

    img_AB = np.concatenate((img_A, img_B), axis=2)
    # img_AB shape: (fine_size, fine_size, input_c_dim + output_c_dim)
    return img_AB

def get_image(image_path, image_size, is_crop=True, resize_w=64, is_grayscale = False):
    return transform(imread(image_path, is_grayscale), image_size, is_crop, resize_w)

def save_images(images, size, image_path):
    return imsave(inverse_transform(images), size, image_path)

def imread(path, is_grayscale = False):
    if (is_grayscale):
        return _imread(path, flatten=True).astype(np.float)
    else:
        return _imread(path, mode='RGB').astype(np.float)

def merge_images(images, size):
    return inverse_transform(images)

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1], 3))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image

    return img

def imsave(images, size, path):
    return scipy.misc.imsave(path, merge(images, size))


def center_crop(x, crop_h, crop_w,
                resize_h=64, resize_w=64):
  if crop_w is None:
    crop_w = crop_h
  h, w = x.shape[:2]
  j = int(round((h - crop_h)/2.))
  i = int(round((w - crop_w)/2.))
  return scipy.misc.imresize(
      x[j:j+crop_h, i:i+crop_w], [resize_h, resize_w])

def transform(image, npx=64, is_crop=True, resize_w=64):

    if is_crop:
        cropped_image = center_crop(image, npx, resize_w=resize_w)
    else:
        cropped_image = image
    return np.array(cropped_image)/127.5 - 1.

def inverse_transform(images):
    return (images+1.)/2.

In [ ]:
from __future__ import division
import os
import time
from glob import glob
import tensorflow as tf
import numpy as np
from collections import namedtuple
import module
from module import *

In [ ]:
!pwd

/content/gdrive/My Drive


In [ ]:
from __future__ import division
import os
import time
from glob import glob
import tensorflow as tf
import numpy as np
from collections import namedtuple

from module import *
#Cyclegan 정의
class cyclegan():
    def __init__(self, sess, checkpoint_dir, test_dir, dataset_dir, which_direction):
        self.sess = sess
        self.batch_size = 1           # 배치 사이즈
        self.image_size = 256            # 잘리는 사이즈
        self.input_c_dim = 3            # 인풋 이미지 채널
        self.output_c_dim = 3          # 아웃풋 이미지 채널
        self.L1_lambda = 10.0
        self.fine_size = 256
        self.ngf = 64         # G의 첫번째 conv 레이어 필터수
        self.ndf = 64         # F의 첫번째 conv 레이어 필터수
        self.output_nc = 3
        self.max_size = 50
        self.beta1 = 0.5        # adam 두번째 옵션
        self.epoch = 200        # 200에폭 돌려라.
        self.epoch_step = 100   # lr를 줄이는 스탭
        self.train_size = 1e8   # 훈련시 이미지 갯수??
        self.lr_init = 0.0002        # 최초의 lr
        self.load_size = 286
        self.save_freq = 500    # 저장 빈도
        self.continue_train = True         # 항상 연속해서 훈련.

        # 경로 모음
        self.checkpoint_dir = checkpoint_dir                # checkpoint 경로.
        self.dataset_dir = dataset_dir              # dataset 경로
        self.test_dir = test_dir                    # 테스트한 이미지를 저장하는 경로.

        # G와 D
        self.discriminator = discriminator
        self.generator = generator_resnet  # resnet을 사용한 generator

        # loss_fn
        self.original_GAN_loss = mae_criterion

        self.which_direction = which_direction

        OPTIONS = namedtuple('OPTIONS', 'batch_size image_size \
                                      gf_dim df_dim output_c_dim')
        self.options = OPTIONS._make((self.batch_size, self.fine_size,
                                      self.ngf, self.ndf, self.output_nc,))       #ngf 는 제너레이터 함수의 첫번째 conv레이어 필터수
                                      #self.phase == 'train'))

        self._build_model()         # 모델 생성.
        #writer = tf.train.SummaryWriter("/tmp/test_logs", sess.graph)
        self.saver = tf.train.Saver()
        self.pool = ImagePool(self.max_size)

    def _build_model(self):
        self.real_data = tf.placeholder(tf.float32,
                                        [None, self.image_size, self.image_size,
                                         self.input_c_dim + self.output_c_dim],     # input 이미지 채널과 output 이미지 채널 수  # 아마 둘다 3. RGB라서.
                                        name='real_A_and_B_images')

        self.real_A = self.real_data[:, :, :, :self.input_c_dim]            # self.real_data의 앞부분, 즉 real A
        self.real_B = self.real_data[:, :, :, self.input_c_dim:self.input_c_dim + self.output_c_dim] # real B
        self.fake_B = self.generator(self.real_A, self.options, False, name="generatorA2B")     # A를 가짜 B로 바꾸기.
        self.fake_A_ = self.generator(self.fake_B, self.options, False, name="generatorB2A")    # 가짜 B를 가짜 A로 바꾸기

        # 앞에 이미 정의한 거라서 reuse를 사용함.
        self.fake_A = self.generator(self.real_B, self.options, True, name="generatorB2A")      # 진짜 B를 가짜 A로 바꾸기
        self.fake_B_ = self.generator(self.fake_A, self.options, True, name="generatorA2B")     # 가짜 A를 가짜 B로 바꾸기
        self.DB_fake = self.discriminator(self.fake_B, self.options, reuse=False, name="discriminatorB")        # 32 BY 32 가 나와..
        self.DA_fake = self.discriminator(self.fake_A, self.options, reuse=False, name="discriminatorA")

        """  G & F 를 학습 시키키 위해 필요한 최종 loss  """
        self.g_loss = self.original_GAN_loss(self.DA_fake, tf.ones_like(self.DA_fake)) \
            + self.original_GAN_loss(self.DB_fake, tf.ones_like(self.DB_fake)) \
            + self.L1_lambda * abs_criterion(self.real_A, self.fake_A_) \
            + self.L1_lambda * abs_criterion(self.real_B, self.fake_B_)

        """    신규 추가 loss   
        #########################################################
        self.A_and_GA_hyunbo = self.generator(self.real_A, self.options, True, name="generatorB2A")
        self.B_and_GB_hyunbo = self.generator(self.real_B, self.options, True, name="generatorA2B")
        self.g_loss_by_hyunbo = self.L1_lambda * abs_criterion(self.real_A, self.A_and_GA_hyunbo) \
                                    + self.L1_lambda * abs_criterion(self.real_B, self.B_and_GB_hyunbo)
        #########################################################"""
        """ =============================== 여기부터는 D의 학습입니다. =========================="""
        # 대문자 D를 함수 처럼 작용하는 듯.
        self.fake_A_sample = tf.placeholder(tf.float32,
                                            [None, self.image_size, self.image_size,
                                             self.input_c_dim], name='fake_A_sample')
        self.fake_B_sample = tf.placeholder(tf.float32,
                                            [None, self.image_size, self.image_size,
                                             self.output_c_dim], name='fake_B_sample')

        # 진짜 B를 넣어서 32 by 32 를 나오게 함.
        self.DB_real = self.discriminator(self.real_B, self.options, reuse=True, name="discriminatorB")
        # 진짜 A를 넣어서 32 by 32 를 나오게 함.
        self.DA_real = self.discriminator(self.real_A, self.options, reuse=True, name="discriminatorA")
        # 가짜 B를 넣어서 32 by 32 를 나오게 함.
        self.DB_fake_sample = self.discriminator(self.fake_B_sample, self.options, reuse=True, name="discriminatorB")
        # 가짜 A를 넣어서 32 by 32 를 나오게 함.
        self.DA_fake_sample = self.discriminator(self.fake_A_sample, self.options, reuse=True, name="discriminatorA")

        """ 진짜 B와 가짜 B를 구별하는 D_loss"""
        self.db_loss_real = self.original_GAN_loss(self.DB_real, tf.ones_like(self.DB_real))
        self.db_loss_fake = self.original_GAN_loss(self.DB_fake_sample, tf.zeros_like(self.DB_fake_sample))
        self.db_loss = (self.db_loss_real + self.db_loss_fake) / 2

        """ 진짜 A와 가짜 A를 구별하는 D_loss """
        self.da_loss_real = self.original_GAN_loss(self.DA_real, tf.ones_like(self.DA_real))
        self.da_loss_fake = self.original_GAN_loss(self.DA_fake_sample, tf.zeros_like(self.DA_fake_sample))
        self.da_loss = (self.da_loss_real + self.da_loss_fake) / 2

        """ D를 학습시키기 위한 최종 loss """
        self.d_loss = self.da_loss + self.db_loss

        """============================================================================================"""

        """ test를 위해 만든 곳 """
        self.test_A = tf.placeholder(tf.float32,
                                     [None, self.image_size, self.image_size,
                                      self.input_c_dim], name='test_A')
        self.test_B = tf.placeholder(tf.float32,
                                     [None, self.image_size, self.image_size,
                                      self.output_c_dim], name='test_B')
        self.testB = self.generator(self.test_A, self.options, True, name="generatorA2B")
        self.testA = self.generator(self.test_B, self.options, True, name="generatorB2A")


        """ D를 학습시킬 때의 변수와 G를 학습시킬 때의 변수를 나눠놓음. """
        t_vars = tf.trainable_variables()
        self.d_vars = [var for var in t_vars if 'discriminator' in var.name]
        self.g_vars = [var for var in t_vars if 'generator' in var.name]
        for var in t_vars: print(var.name)

    def train(self):
        """Train cyclegan"""
        self.lr_var = tf.placeholder(tf.float32, None, name='learning_rate')
        self.d_optim = tf.train.AdamOptimizer(self.lr_var, beta1=self.beta1) \
            .minimize(self.d_loss, var_list=self.d_vars)
        self.g_optim = tf.train.AdamOptimizer(self.lr_var, beta1=self.beta1) \
            .minimize(self.g_loss, var_list=self.g_vars)

        """
        #########################################################
        self.hyunbo_optim = tf.train.AdamOptimizer(self.lr_var, beta1=self.beta1) \
            .minimize(self.g_loss_by_hyunbo, var_list=self.g_vars)
        #########################################################
        """

        init_op = tf.global_variables_initializer()     #모든 변수 초기화.
        self.sess.run(init_op)

        counter = 1
        start_time = time.time()

        if self.continue_train:
            suc_or_fal, num_of_train = self.load()          # 성공, 실패 유무 와 훈련횟수
            if suc_or_fal:
                print(" check point 가져오기 성공")
                counter = int(num_of_train.split('-')[1])                     # 이어서 저장하기 위해서..
            else:
                print(" check point 가져오기 실패")

        for epoch in range(self.epoch):
            dataA = glob('{}/*.*'.format(self.dataset_dir + '/trainA'))
            dataB = glob('{}/*.*'.format(self.dataset_dir + '/trainB'))
            print("num of dataA : ", dataA.__len__())
            np.random.shuffle(dataA)
            np.random.shuffle(dataB)
            batch_idxs = min(min(len(dataA), len(dataB)), self.train_size) // self.batch_size
            lr = self.lr_init if epoch < self.epoch_step else self.lr_init*(self.epoch-epoch)/(self.epoch-self.epoch_step)    # lr이 작아짐. 100에폭이 지나면.

            for idx in range(0, batch_idxs):
                batch_files = list(zip(dataA[idx * self.batch_size:(idx + 1) * self.batch_size],
                                       dataB[idx * self.batch_size:(idx + 1) * self.batch_size]))
                batch_images = [load_train_data(batch_file, self.load_size, self.fine_size) for batch_file in batch_files]      # 이미지 크기 바꾸기.
                batch_images = np.array(batch_images).astype(np.float32)

                # Update G network and record fake outputs
                fake_A, fake_B, _ = self.sess.run(
                    [self.fake_A, self.fake_B, self.g_optim],
                    feed_dict={self.real_data: batch_images, self.lr_var: lr})

                #self.writer.add_summary(summary_str, counter)
                #[fake_A, fake_B] = self.pool([fake_A, fake_B])

                # Update D network
                self.sess.run( [self.d_optim],
                               feed_dict={self.real_data: batch_images,
                                    self.fake_A_sample: fake_A,
                                    self.fake_B_sample: fake_B,
                                    self.lr_var: lr})
                #self.writer.add_summary(summary_str, counter)

                counter += 1
                print(("Epoch: [%2d] [%4d/%4d] time: %4.4f" % (
                    epoch, idx, batch_idxs, time.time() - start_time)))

                # 저장.
                if np.mod(counter, self.save_freq) == 2:
                    self.save(counter)
                    print("ckpt 저장 완료")

    def save(self, step):

        step_str = str(step)

        self.saver.save(self.sess,
                        self.checkpoint_dir + "/" + step_str,
                        global_step=step)

        """ 여기는 ckpt 파일을 gdrive에 올리는 코드입니다."""
        """latest = tf.train.latest_checkpoint(checkpoint_dir)
        file_name = latest.split("/")
        file_name = file_name[-1]
        list = ['.data-00000-of-00001', '.index', '.meta']
        for x in list:
            drive_service = build('drive', 'v3')
            file_metadata = {
                'name': file_name + x,  # 구글에 올라가는 이름
                'mimeType': None
            }
            media = MediaFileUpload('' + latest + x,
                                    mimetype=None,
                                    resumable=True)
            created = drive_service.files().create(body=file_metadata,
                                                   media_body=media,
                                                   fields='id').execute()
            print('File ID: {}'.format(created.get('id')))
            print(x + "gdrive에 올리기 성공")"""

    def load(self):
        print(" [*] Reading checkpoint...")

        #model_dir = "%s_%s" % (self.dataset_dir, self.image_size)
        #checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

        ckpt = tf.train.get_checkpoint_state(self.checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(self.checkpoint_dir, ckpt_name))
            print(" 체크 포인트 이름!! : ", ckpt_name)
            index = ckpt_name.find("-")
            num_of_train = ckpt_name[index + 1:]
            print(" 불러온 ckpt 횟수 : ", num_of_train)
            time.sleep(10)
            return True, num_of_train
        else:
            print("check point가 경로에 없습니다.")
            return False, 1             # 리턴값 맞춰주기 위해 씀. 아무값이나 리턴

    def test(self):
        """Test cyclegan"""
        init_op = tf.global_variables_initializer()
        self.sess.run(init_op)
        if self.which_direction == 'AtoB':
            sample_files = glob('{}/*.*'.format(self.dataset_dir + '/testA'))
            print("dataset 가져오기 성공.")
        elif self.which_direction == 'BtoA':
            sample_files = glob('{}/*.*'.format(self.dataset_dir + '/testB'))
            print("dataset 가져오기 성공.")
        else:
            raise Exception('AtoB BtoA 둘중 하나는 선택하세요.')

        if self.load():
            print(" check point 가져오기 성공")
        else:
            print(" check point 가져오기 실패")

        if self.which_direction == 'AtoB':
            out_var, in_var = (self.testB, self.test_A)
        else:
            out_var, in_var = (self.testA, self.test_B)

        for sample_file in sample_files:
            print('Processing image: ' + sample_file)
            sample_image = [load_test_data(sample_file, self.fine_size)]
            sample_image = np.array(sample_image).astype(np.float32)
            image_path = os.path.join(self.test_dir,
                                      '{0}_{1}'.format(self.which_direction, os.path.basename(sample_file)))
            fake_img = self.sess.run(out_var, feed_dict={in_var: sample_image})
            print("start saving")
            save_images(fake_img, [1, 1], image_path)


def main():
    checkpoint_dir = './checkpoint/face_256'             # 체크포인트 경로
    test_dir = './test'                          # 테스트 이미지가 저장되는 경로
    dataset_dir = './datasets'                         # 데이터셋 위치    trainA trainB testA testB
    phase = "test"  # or test
    which_direction = "BtoA"        # or BtoA .  테스트시 변환 방향
    # 이미지 저장은 test 경로를 따로 만들어서 함.

    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)

    tfconfig = tf.ConfigProto(allow_soft_placement=True)        # gpu를 사용하겠습니다.
    tfconfig.gpu_options.allow_growth = True

    with tf.Session(config=tfconfig) as sess:
        model = cyclegan(sess, checkpoint_dir = checkpoint_dir, test_dir = test_dir,\
                         dataset_dir= dataset_dir, which_direction= which_direction)
        if phase == 'train':
            print(" 훈련 시작")
            model.train()
        elif phase == "test":
            print(" 테스트 시작")
            model.test()
        else:
            print(" train??? test???? 둘중하나는 고르세요.")

if __name__ == '__main__':
    main()

ValueError: ignored